# ResNet-18 with Gaussian Blur Curriculum Learning on ImageNet-100

This notebook implements ResNet-18 with Gaussian blur curriculum learning on ImageNet-100, based on the approach from the CVT_13_blur notebook.

## Curriculum Learning Strategy
- **Early epochs**: Train with Gaussian blur applied to images
- **Later epochs**: Train with original sharp images
- **Blur parameters**: Kernel size 7, sigma 1.0, applied for first 20 epochs

## Dataset: ImageNet-100
- 100 classes subset of ImageNet
- 224x224 input images
- Standard data augmentation with blur curriculum

## Model: ResNet-18
- Standard ResNet-18 architecture
- ~11.7M parameters
- Curriculum learning with Gaussian blur


## 1. Setup and Imports


In [ ]:
%pip install torchsummary torchvision tqdm wandb


In [4]:
!pip freeze > requirements.txt


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import os
import gc
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from datetime import datetime
import wandb

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


## 2. Configuration


In [ ]:
# Training configuration with blur curriculum
config = {
    'dataset_path': 'IDL_data/ImageNet100_224',
    'batch_size': 128,
    'num_epochs': 100,
    'learning_rate': 0.01,
    'weight_decay': 1e-4,
    'momentum': 0.9,
    'num_classes': 100,
    'image_size': 224,
    'num_workers': 8,
    'save_dir': './checkpoints_blur_curriculum',
    'use_wandb': True,
    'project_name': 'resnet18-imagenet100-blur-curriculum',
    'run_name': 'blur-curriculum-training',
    'BLUR': {
        'KERNEL_SIZE': 7,
        'SIGMA': 1.0,
        'EPOCHS': 20  # Number of epochs to use blur
    }
}

print("Configuration:")
for key, value in config.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for sub_key, sub_value in value.items():
            print(f"    {sub_key}: {sub_value}")
    else:
        print(f"  {key}: {value}")


## 3. ResNet-18 Model Definition


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # First conv layer for ImageNet (224x224 input)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # ResNet layers
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        # Global average pooling and classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


def ResNet18(num_classes=100):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Test model creation
model = ResNet18(num_classes=config['num_classes'])
total_params = sum(p.numel() for p in model.parameters())
print(f"ResNet-18 created with {total_params:,} parameters")
print(f"Model size: {total_params/1e6:.2f}M parameters")


In [ ]:
# Update model to return dictionary format for experiment compatibility
class ResNetExperiment(ResNet):
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return {"out": out}

# Create model with experiment format
model = ResNetExperiment(BasicBlock, [2, 2, 2, 2], num_classes=config['num_classes'])
total_params = sum(p.numel() for p in model.parameters())
print(f"ResNet-18 (Experiment format) created with {total_params:,} parameters")
print(f"Model size: {total_params/1e6:.2f}M parameters")


## 4. Data Loading and Preprocessing with Blur Curriculum


In [ ]:
def get_imagenet100_loaders_with_blur(data_path, batch_size=128, num_workers=5, blur_config=None):
    """Load ImageNet-100 dataset with blur curriculum"""
    
    # ImageNet normalization
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
    
    # Training transforms with augmentation (sharp images)
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        normalize,
    ])
    
    # Training transforms with Gaussian blur for curriculum
    transform_train_blur = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.GaussianBlur(
            kernel_size=blur_config['KERNEL_SIZE'] if blur_config else 7,
            sigma=blur_config['SIGMA'] if blur_config else 1.0
        ),
        normalize,
    ])
    
    # Validation transforms (no augmentation)
    transform_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
    
    # Load datasets
    train_dir = Path(data_path) / 'train'
    val_dir = Path(data_path) / 'val'
    
    train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
    train_dataset_blur = datasets.ImageFolder(train_dir, transform=transform_train_blur)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform_val)
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=True,
        drop_last=True
    )
    
    train_loader_blur = DataLoader(
        train_dataset_blur, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, 
        pin_memory=True,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=num_workers, 
        pin_memory=True
    )
    
    return train_loader, train_loader_blur, val_loader, train_dataset, val_dataset

# Load the dataset
print("Loading ImageNet-100 dataset with blur curriculum...")
train_loader, train_loader_blur, val_loader, train_dataset, val_dataset = get_imagenet100_loaders_with_blur(
    data_path=config['dataset_path'],
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    blur_config=config['BLUR']
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Number of classes: {len(train_dataset.classes)}")
print(f"Classes: {train_dataset.classes[:10]}...")
print(f"Training batches: {len(train_loader)}")
print(f"Training blur batches: {len(train_loader_blur)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Blur curriculum: First {config['BLUR']['EPOCHS']} epochs with blur, then sharp images")


In [10]:
def train_epoch_with_curriculum(model, train_loader, train_loader_blur, criterion, optimizer, device, epoch, config):
    """Train for one epoch with blur curriculum"""
    model.train()
    
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    # Choose loader based on curriculum
    if epoch < config['BLUR']['EPOCHS']:
        loader = train_loader_blur
        curriculum_type = "Blur"
    else:
        loader = train_loader
        curriculum_type = "Sharp"
    
    pbar = tqdm(loader, desc=f'Epoch {epoch+1} ({curriculum_type})')
    
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # Update metrics
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), data.size(0))
        top1.update(acc1[0], data.size(0))
        top5.update(acc5[0], data.size(0))
        
        # Update progress bar
        pbar.set_postfix({
            'Loss': f'{losses.avg:.4f}',
            'Acc@1': f'{top1.avg:.2f}%',
            'Acc@5': f'{top5.avg:.2f}%',
            'Type': curriculum_type
        })
    
    return losses.avg, top1.avg, top5.avg


## 5. Training and Evaluation Functions


In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            loss = criterion(output, target)
            
            # Update metrics
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), data.size(0))
            top1.update(acc1[0], data.size(0))
            top5.update(acc5[0], data.size(0))
            
            pbar.set_postfix({
                'Loss': f'{losses.avg:.4f}',
                'Acc@1': f'{top1.avg:.2f}%',
                'Acc@5': f'{top5.avg:.2f}%'
            })
    
    return losses.avg, top1.avg, top5.avg


def adjust_learning_rate(optimizer, epoch, initial_lr, lr_decay_epochs=[30, 60, 90]):
    """Decay learning rate by 10 at specified epochs"""
    lr = initial_lr
    for decay_epoch in lr_decay_epochs:
        if epoch >= decay_epoch:
            lr *= 0.1
    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    return lr

print("Training and evaluation functions defined.")


## 6. Initialize Model, Optimizer, and Loss


In [ ]:
# Initialize model
model = ResNet18(num_classes=config['num_classes']).to(DEVICE)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer (SGD with momentum)
optimizer = optim.SGD(
    model.parameters(), 
    lr=config['learning_rate'], 
    momentum=config['momentum'], 
    weight_decay=config['weight_decay']
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

print(f"Model initialized on {DEVICE}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Initial learning rate: {config['learning_rate']}")
print(f"Optimizer: SGD with momentum {config['momentum']}")
print(f"Weight decay: {config['weight_decay']}")
print(f"Blur curriculum: {config['BLUR']['EPOCHS']} epochs with blur, then sharp images")


## 7. Initialize Weights & Biases (Optional)


In [ ]:
wandb.login(key=os.environ.get('WANDB_API_KEY')) # API Key is in your wandb account, under settings (wandb.ai/settings)


In [ ]:
# Create your wandb run
run = wandb.init(
    name = "ResNet18 ImageNet100 Blur Curriculum", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "imagenet100-blur-curriculum", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)


## 8. Create Checkpoint Directory


In [ ]:
# Create checkpoint directory
os.makedirs(config['save_dir'], exist_ok=True)
print(f"Checkpoint directory created: {config['save_dir']}")


## 9. Main Training Loop with Blur Curriculum


## Main Training Loop (Resumable)


In [ ]:
# Check for existing checkpoint and resume if found
last_path = os.path.join(config['save_dir'], 'last.pth')
best_path = os.path.join(config['save_dir'], 'best_model.pth')

start_epoch = 0
best_acc1 = 0.0
best_epoch = 0

if os.path.exists(last_path):
    print(f"Found checkpoint at {last_path}. Resuming...")
    try:
        ckpt = torch.load(last_path, map_location=DEVICE)
        model.load_state_dict(ckpt['model_state_dict'])
        optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        scheduler.load_state_dict(ckpt['scheduler_state_dict'])
        start_epoch = int(ckpt.get('epoch', 0)) + 1
        best_acc1 = float(ckpt.get('best_acc1', 0.0))
        print(f"Resumed from epoch {start_epoch} with best_acc1={best_acc1:.2f}%")
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        print("Starting fresh training...")
        start_epoch = 0
        best_acc1 = 0.0
else:
    print("No existing checkpoint. Starting fresh.")

# Training history
history = {
    'train_loss': [],
    'train_acc1': [],
    'train_acc5': [],
    'val_loss': [],
    'val_acc1': [],
    'val_acc5': [],
    'learning_rate': [],
    'curriculum_type': []  # Track whether blur or sharp was used
}

print(f"Starting training for {config['num_epochs']} epochs...")
print(f"Blur curriculum: First {config['BLUR']['EPOCHS']} epochs with blur, then sharp images")
print(f"Training on {len(train_dataset)} samples")
print(f"Validation on {len(val_dataset)} samples")
print("="*60)

for epoch in range(start_epoch, config['num_epochs']):
    print(f"\nEpoch {epoch+1}/{config['num_epochs']}")
    
    # Determine curriculum type
    curriculum_type = "Blur" if epoch < config['BLUR']['EPOCHS'] else "Sharp"
    print(f"Curriculum: {curriculum_type} images")
    
    # Adjust learning rate
    current_lr = adjust_learning_rate(optimizer, epoch, config['learning_rate'])
    
    # Train for one epoch with curriculum
    train_loss, train_acc1, train_acc5 = train_epoch_with_curriculum(
        model, train_loader, train_loader_blur, criterion, optimizer, DEVICE, epoch, config
    )
    
    # Validate
    val_loss, val_acc1, val_acc5 = validate(model, val_loader, criterion, DEVICE)
    
    # Update scheduler
    scheduler.step()
    
    # Save 'last' checkpoint every epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_acc1': best_acc1,
        'val_acc1': val_acc1,
        'val_acc5': val_acc5,
    }, last_path)
    
    # Track best and save best model
    if val_acc1 > best_acc1:
        best_acc1 = val_acc1
        best_epoch = epoch + 1
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_acc1': best_acc1,
            'val_acc1': val_acc1,
            'val_acc5': val_acc5,
        }, best_path)
        print(f"New best model saved! Acc@1: {best_acc1:.2f}%")
    
    # Save metrics (convert tensors to floats for JSON serialization)
    history['train_loss'].append(float(train_loss))
    history['train_acc1'].append(float(train_acc1))
    history['train_acc5'].append(float(train_acc5))
    history['val_loss'].append(float(val_loss))
    history['val_acc1'].append(float(val_acc1))
    history['val_acc5'].append(float(val_acc5))
    history['learning_rate'].append(float(current_lr))
    history['curriculum_type'].append(curriculum_type)
    
    # Log to wandb
    if run is not None:
        run.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc1': train_acc1,
            'train_acc5': train_acc5,
            'val_loss': val_loss,
            'val_acc1': val_acc1,
            'val_acc5': val_acc5,
            'learning_rate': current_lr,
            'curriculum_type': curriculum_type
        })
    
    # Print epoch results
    print(f"Train Loss: {train_loss:.4f} | Train Acc@1: {train_acc1:.2f}% | Train Acc@5: {train_acc5:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc@1: {val_acc1:.2f}% | Val Acc@5: {val_acc5:.2f}%")
    print(f"Learning Rate: {current_lr:.6f}")
    print(f"Curriculum: {curriculum_type}")
    print("-" * 60)

print(f"\nTraining completed!")
print(f"Best validation accuracy: {best_acc1:.2f}% (Epoch {best_epoch})")


## 10. Save Final Results


In [ ]:
# Save final model and history
final_checkpoint = {
    'epoch': config['num_epochs'],
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'best_acc1': best_acc1,
    'best_epoch': best_epoch,
    'history': history,
    'config': config
}

torch.save(final_checkpoint, os.path.join(config['save_dir'], 'final_model.pth'))

# Save training history as JSON (convert any remaining tensors to floats)
def convert_to_serializable(obj):
    """Convert tensors and other non-serializable objects to Python types"""
    if isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif hasattr(obj, 'item'):  # PyTorch tensor
        return obj.item()
    elif hasattr(obj, 'tolist'):  # NumPy array
        return obj.tolist()
    else:
        return obj

# Convert history to JSON-serializable format
serializable_history = convert_to_serializable(history)

with open(os.path.join(config['save_dir'], 'training_history.json'), 'w') as f:
    json.dump(serializable_history, f, indent=2)

print(f"Final results saved to {config['save_dir']}")
print(f"Best model: {os.path.join(config['save_dir'], 'best_model.pth')}")
print(f"Final model: {os.path.join(config['save_dir'], 'final_model.pth')}")
print(f"Training history: {os.path.join(config['save_dir'], 'training_history.json')}")


## 12. Final Results Summary


In [ ]:
print("="*70)
print("FINAL TRAINING RESULTS - BLUR CURRICULUM")
print("="*70)
print(f"Dataset: ImageNet-100")
print(f"Model: ResNet-18")
print(f"Training Strategy: Gaussian Blur Curriculum Learning")
print(f"Blur Curriculum: First {config['BLUR']['EPOCHS']} epochs with blur, then sharp images")
print(f"Blur Parameters: Kernel size {config['BLUR']['KERNEL_SIZE']}, Sigma {config['BLUR']['SIGMA']}")
print(f"Total Epochs: {config['num_epochs']}")
print(f"Batch Size: {config['batch_size']}")
print(f"Initial Learning Rate: {config['learning_rate']}")
print(f"Weight Decay: {config['weight_decay']}")
print(f"Momentum: {config['momentum']}")
print("-"*70)
print(f"Best Validation Accuracy: {best_acc1:.2f}% (Epoch {best_epoch})")
print(f"Final Training Accuracy: {history['train_acc1'][-1]:.2f}%")
print(f"Final Validation Accuracy: {history['val_acc1'][-1]:.2f}%")
print(f"Final Training Top-5 Accuracy: {history['train_acc5'][-1]:.2f}%")
print(f"Final Validation Top-5 Accuracy: {history['val_acc5'][-1]:.2f}%")
print("-"*70)
print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Model Size: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
print("="*70)

# Close wandb run if active
if run is not None:
    run.finish()
    print("Wandb run finished.")


## 13. Load and Test Best Model (Optional)


In [ ]:
# Load best model for testing
best_model_path = os.path.join(config['save_dir'], 'best_model.pth')
if os.path.exists(best_model_path):
    print("Loading best model for final evaluation...")
    
    # Load checkpoint
    checkpoint = torch.load(best_model_path)
    
    # Create model and load weights
    best_model = ResNet18(num_classes=config['num_classes']).to(DEVICE)
    best_model.load_state_dict(checkpoint['model_state_dict'])
    
    # Evaluate on validation set
    print("Evaluating best model on validation set...")
    val_loss, val_acc1, val_acc5 = validate(best_model, val_loader, criterion, DEVICE)
    
    print(f"Best Model Results:")
    print(f"  Validation Loss: {val_loss:.4f}")
    print(f"  Validation Acc@1: {val_acc1:.2f}%")
    print(f"  Validation Acc@5: {val_acc5:.2f}%")
    print(f"  Best epoch: {checkpoint['epoch']}")
else:
    print("Best model checkpoint not found.")


## 14. Cleanup and Summary


In [ ]:
# Clean up GPU memory
gc.collect()
torch.cuda.empty_cache()

print("Training completed successfully!")
print(f"\nCheckpoint directory: {config['save_dir']}")
print("Files created:")
for file in os.listdir(config['save_dir']):
    file_path = os.path.join(config['save_dir'], file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path) / (1024*1024)  # Size in MB
        print(f"  - {file} ({size:.1f} MB)")

print("\nThis blur curriculum implementation can be compared with the baseline ResNet-18.")
print("Key differences from baseline:")
print(f"- First {config['BLUR']['EPOCHS']} epochs use Gaussian blur (kernel={config['BLUR']['KERNEL_SIZE']}, sigma={config['BLUR']['SIGMA']})")
print("- Remaining epochs use sharp images")
print("- Same model architecture and hyperparameters as baseline")
print("- Curriculum learning strategy for improved training")
